In [4]:
import numpy as np
import json
import os
from scipy.special import expit
#import cv2
import requests

import time
#import BaseHTTPServer

from magenta.models.sketch_rnn.sketch_rnn_train import *
from magenta.models.sketch_rnn.model import *
from magenta.models.sketch_rnn.utils import *
from magenta.models.sketch_rnn.rnn import *


HOST_NAME = 'localhost' # !!!REMEMBER TO CHANGE THIS!!!
PORT_NUMBER = 9020 # Maybe set this to 9000.

import uuid

'''
class MyHandler(BaseHTTPServer.BaseHTTPRequestHandler):
    def do_HEAD(s):
        s.send_response(200)
        s.send_header("Content-type", "application/json")
        s.end_headers()
    def do_GET(s):
        """Respond to a GET request."""
        s.send_response(200)
        s.send_header("Content-type", "application/json")
        s.end_headers()
        lines = get_sketch()
        s.wfile.write(json.dumps(lines))
'''

#server_class = BaseHTTPServer.HTTPServer
#httpd = server_class((HOST_NAME, PORT_NUMBER), MyHandler)

class SketchLSTMCell(object):

    def __init__(self, num_units, input_size, Wxh, Whh, bias):
        self.num_units = num_units;
        self.input_size = input_size;
        self.Wxh = Wxh;
        self.Whh = Whh;
        self.bias = bias;
        self.forget_bias = 1.0;
        self.Wfull=np.transpose(np.concatenate((np.transpose(Wxh), np.transpose(Whh)),axis=1))

    def get_pdf(self,s, dec_output_w_, dec_output_b_):
        h = s[0];
        NOUT = N_mixture;
        #print(dec_output_w)
        z = np.dot(h, dec_output_w_) + dec_output_b_;
        z_pen_logits = z[0:3];
        z_pi = z[3+NOUT*0:3+NOUT*1];
        z_mu1 = z[3+NOUT*1:3+NOUT*2];
        z_mu2 = z[3+NOUT*2:3+NOUT*3];
        z_sigma1 = np.exp(z[3+NOUT*3:3+NOUT*4]);
        z_sigma2 = np.exp(z[3+NOUT*4:3+NOUT*5]);
        z_corr = np.tanh(z[3+NOUT*5:3+NOUT*6]);
        z_pen = softmax(z_pen_logits)
        z_pi = softmax(z_pi);
        return [z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_pen];

    def zero_state(self):
        return [np.zeros(self.num_units), np.zeros(self.num_units)]

    def __call__(self,x,h,c):
        concat = np.concatenate((x, h));
        d = np.dot(concat, self.Wfull)
        hidden = d + self.bias
        num_units = self.num_units;
        forget_bias = self.forget_bias;
        i = expit(hidden[0*num_units:1*num_units]);
        g = np.tanh(hidden[1*num_units:2*num_units]);
        f = expit(hidden[2*num_units:3*num_units] + forget_bias);
        o = expit(hidden[3*num_units:4*num_units]);

        new_c = c*f + g*i
        new_h = np.tanh(new_c) * o;
        return [new_h, new_c];

#
#
#   model file
#
#


model_dir = '/Users/mhy/Desktop/data lab/magenta/magenta/models/sketch_rnn/test_sketcher/weights-uncon-7500'

[hps_model, eval_hps_model, sample_hps_model] = load_model(model_dir)

# construct the sketch-rnn model here:
reset_graph()

'''
print("=============== hps_model =================")
print(hps_model)
print("=============== hps_model =================")
print("=============== eval_hps_model =================")
print(eval_hps_model)
print("=============== eval_hps_model =================")
print("=============== sample_hps_model =================")
print(sample_hps_model)
print("=============== sample_hps_model =================")
'''

model = Model(hps_model)
eval_model = Model(eval_hps_model, reuse=True)
sample_model = Model(sample_hps_model, reuse=True)

'''
print("\n\n\n\n\n")
print("+++++++++++++++ ____model____ +++++++++++++++++")
print(model)
print("+++++++++++++++ eval_____model ++++++++++++++++++")
print("+++++++++++++++ eval____model +++++++++++++++++")
print(eval_model)
print("+++++++++++++++ sample_____model ++++++++++++++++++")
print("+++++++++++++++ sample____model +++++++++++++++++")
print(sample_model)
print("+++++++++++++++ sample_____model ++++++++++++++++++")
'''

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# loads the weights from checkpoint into our model
load_checkpoint(sess, model_dir)
v = tf.trainable_variables()
x = sess.run(v)

output_w_ = [v for v in tf.trainable_variables() if v.name == "vector_rnn/RNN/output_w:0"][0].eval()
output_b_ = [v for v in tf.trainable_variables() if v.name == "vector_rnn/RNN/output_b:0"][0].eval()
lstm_W_xh_ = [v for v in tf.trainable_variables() if v.name == "vector_rnn/RNN/LSTMCell/W_xh:0"][0].eval()
lstm_W_hh_ = [v for v in tf.trainable_variables() if v.name == "vector_rnn/RNN/LSTMCell/W_hh:0"][0].eval()
lstm_bias_ = [v for v in tf.trainable_variables() if v.name == "vector_rnn/RNN/LSTMCell/bias:0"][0].eval()

#print(type(output_w_))
#print(type(output_b_))
#print(type(lstm_W_xh_))
#print(type(lstm_W_hh_))
#print(type(lstm_bias_))

dec_output_w = output_w_;
dec_output_b = output_b_;
dec_lstm_W_xh = lstm_W_xh_;
dec_lstm_W_hh = lstm_W_hh_;
dec_lstm_bias = lstm_bias_;
dec_num_units = dec_lstm_W_hh.shape[0];
dec_input_size = dec_lstm_W_xh.shape[0];

print("===============")
#print(dec_output_w.shape)
#print(dec_output_b.shape)
#print(dec_lstm_W_xh.shape)
#print(dec_lstm_W_hh.shape)
#print(dec_lstm_bias.shape)
#print(dec_output_w)
print("===============")

dec_lstm = SketchLSTMCell(dec_num_units, dec_input_size, dec_lstm_W_xh, dec_lstm_W_hh, dec_lstm_bias)

N_mixture = 20
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sample(z, temperature=None, softmax_temperature=None):
    temp=0.25;
    if temperature is not None:
        temp = temperature;
    softmax_temp = 0.5+temp*0.5;
    if softmax_temperature is not None:
        softmax_temp = softmax_temperature;
    z_0 = adjust_temp(z[0], softmax_temp);
    z_6 = adjust_temp(z[6], softmax_temp);
    idx = sample_softmax(z_0);
    mu1 = z[1][idx];
    mu2 = z[2][idx];
    sigma1 = z[3][idx]*np.sqrt(temp);
    sigma2 = z[4][idx]*np.sqrt(temp);
    corr = z[5][idx];
    pen_idx = sample_softmax(z_6);
    penstate = [0, 0, 0];
    penstate[pen_idx] = 1;
    delta = birandn(mu1, mu2, sigma1, sigma2, corr);
    return [delta[0]*scale_factor, delta[1]*scale_factor, penstate[0], penstate[1], penstate[2]];

def adjust_temp(z_old, temp):
    z = z_old.copy();
    z = np.log(z)/temp
    x = z.max()
    z = z - x
    z = np.exp(z)
    x = z.sum()
    z = z/x
    return z;

def randf(a, b):
    return np.random.random()*(b-a)+a

def sample_softmax(z_sample):
    x = randf(0,1)
    N = z_sample.shape[0]
    accumulate = 0
    for i in range(N):
        accumulate += z_sample[i];
        if accumulate >= x:
            return i
    return -1;
return_v = False;
v_val = 0.0;

def gaussRandom():
    global return_v, v_val
    if return_v:
        return_v = False;
        return v_val;

    u = 2*np.random.random()-1;
    v = 2*np.random.random()-1;
    r = u*u + v*v;
    if r == 0 or  r > 1:
        return gaussRandom();
    c = np.sqrt(-2*np.log(r)/r);
    v_val = v*c;
    return_v = True;
    return u*c;

def randn(mu, std):
    return mu+gaussRandom()*std

def birandn(mu1, mu2, std1, std2, rho):
    z1 = randn(0,1);
    z2 = randn(0,1);
    x = np.sqrt(1-rho*rho)*std1*z1 + rho*std1*z2 + mu1;
    y = std2*z2 + mu2;
    return [x, y];

def generate(dec_lstm_, dec_output_w_, dec_output_b_, temperature = None, softmax_temperature = None):
    temp=0.25;
    if temperature is not None:
        temp = temperature;
    softmax_temp = 0.5+temp*0.5;
    if softmax_temperature is not None:
        softmax_temp = softmax_temperature
    init_state = dec_lstm_.zero_state()
    h = init_state[0];
    c = init_state[1];

    x = np.array([0, 0, 0, 0, 0],dtype=np.float)
    result = [];
    max_seq_len = 125
    for i in range(max_seq_len):
        lstm_input = x;
        rnn_state = dec_lstm_(lstm_input, h, c);
        pdf = dec_lstm.get_pdf(rnn_state, dec_output_w_, dec_output_b_)
        [dx, dy, pen_down, pen_up, pen_end] = sample(pdf, temp, softmax_temp);
        result.append([dx, dy, pen_down, pen_up, pen_end]);
        if pen_end == 1:
            return result;
        x = np.array([dx/scale_factor, dy/scale_factor, pen_down, pen_up, pen_end]);

        h = rnn_state[0];
        c = rnn_state[1];
    result.append([0, 0, 0, 0, 1]);
    return result
max_seq_len = 123
scale_factor = 99.698

def get_sketch(sketch):
    #sketch = generate()
    #print(sketch)
    xsum, ysum = [0],[0]
    for i,l in enumerate(sketch):
        xsum.append(l[0]+xsum[i])
        ysum.append(l[1]+ysum[i])
    width = np.max(xsum) - np.min(xsum)
    height = np.max(ysum) - np.min(ysum)
    drawing = np.zeros((int(height)+40,int(width)+40,3),dtype=np.uint8)
    drawing[:,:] = (255,255,255)
    x_start, y_start = 20-int(np.min(xsum)),20-int(np.min(ysum))

    x, y = x_start,y_start
    prev_pen = [1, 0, 0]

    x_lines = []
    y_lines = []
    for l in sketch:
        dx, dy, pen_down, pen_up, pen_end = l
        dx = int(dx)
        dy = int(dy)
        if prev_pen[2] == 1:
            break
        if prev_pen[0] == 1:
            x_lines.extend([x,x+dx])
            y_lines.extend([y,y+dy])
            #lines.append([[x,y],[x+dx,y+dy]])
        x += dx;
        y += dy;
        prev_pen = [pen_down, pen_up, pen_end]
    return [x_lines,y_lines]



INFO:tensorflow:Model using gpu.
INFO:tensorflow:Input dropout mode = False.
INFO:tensorflow:Output dropout mode = False.
INFO:tensorflow:Recurrent dropout mode = True.
INFO:tensorflow:Model using gpu.
INFO:tensorflow:Input dropout mode = 0.
INFO:tensorflow:Output dropout mode = 0.
INFO:tensorflow:Recurrent dropout mode = 0.
INFO:tensorflow:Model using gpu.
INFO:tensorflow:Input dropout mode = 0.
INFO:tensorflow:Output dropout mode = 0.
INFO:tensorflow:Recurrent dropout mode = 0.
INFO:tensorflow:Loading model /Users/mhy/Desktop/data lab/magenta/magenta/models/sketch_rnn/test_sketcher/weights-uncon-7500/vector-7500.
INFO:tensorflow:Restoring parameters from /Users/mhy/Desktop/data lab/magenta/magenta/models/sketch_rnn/test_sketcher/weights-uncon-7500/vector-7500


In [5]:
num_of_boats = 10000
count = 0
train_result = []
for j in range(num_of_boats):
    output = []
    result = generate(dec_lstm, dec_output_w, dec_output_b)
    total = len(result)
    hostname = "54.82.94.146"
    port = 80
    check = 0
    x_array,y_array = get_sketch(result)
    print(x_array)
    #rint(y_array)
    r = requests.post("http://{}:{}/Hdata".format(hostname,port),
                        data = json.dumps({"data":{"x_data":x_array,"y_data":y_array,"id":j,"check":check}}))

[688, 749, 749, 863, 863, 933, 933, 957, 957, 983, 983, 998, 998, 1022, 1022, 972, 972, 898, 898, 847, 847, 800, 800, 732, 732, 703, 703, 688, 688, 659, 659, 649, 649, 657, 673, 674, 674, 729, 729, 739, 739, 740, 740, 730, 730, 725, 725, 725, 591, 540, 540, 450, 450, 366, 366, 342, 342, 300, 300, 278, 278, 261, 261, 237, 237, 225, 225, 216, 216, 142, 142, 108, 108, 90, 90, 51, 51, 32, 32, 27, 930, 949, 949, 977, 977, 1082, 1082, 1131, 1131, 1211, 1211, 1264, 1264, 1311, 1311, 1357, 1357, 1498, 1498, 1550, 1550, 1582, 1582, 1602, 1602, 1731, 1731, 1774, 690, 701, 701, 718, 718, 732, 732, 744, 744, 758, 758, 784, 784, 796, 817, 815, 815, 819, 815, 816, 830, 834, 853, 868, 860, 866, 867, 886, 867, 862, 881, 896, 881, 886, 873, 866, 883, 880, 894, 894, 894, 894, 894, 899, 899, 904, 904, 906, 906, 885, 885, 878, 878, 880, 880, 875, 875, 877, 877, 884, 884, 890, 890, 896, 896, 896, 896, 886, 886, 885, 885, 891, 891, 891, 891, 886]
[1007, 951, 951, 932, 932, 864, 864, 782, 782, 685, 685, 585,

[38, 66, 66, 184, 184, 254, 254, 407, 407, 503, 503, 507, 507, 555, 555, 575, 575, 346, 346, 169, 169, 52, 52, 18, 613, 665, 665, 675, 675, 669, 669, 625, 625, 612, 612, 613, 613, 634, 634, 645, 645, 653, 653, 667, 667, 673, 673, 649, 649, 648, 648, 661, 661, 676, 676, 657, 657, 633, 633, 635, 635, 656, 656, 665, 665, 669, 669, 664, 664, 643, 643, 639, 288, 340, 340, 364, 364, 449]
[53, 90, 90, 203, 203, 251, 251, 488, 488, 640, 640, 747, 747, 813, 813, 856, 856, 880, 880, 888, 888, 885, 885, 860, 860, 818, 818, 771, 771, 599, 599, 372, 372, 199, 199, 91, 91, 36, 36, 17, 17, 27, 27, 42, 270, 302, 302, 317, 317, 353, 353, 372, 372, 413, 413, 445, 445, 478, 478, 509, 509, 551, 551, 607, 607, 620, 620, 618, 355, 343, 343, 340, 340, 345, 345, 355, 355, 467, 467, 517, 517, 530, 530, 537, 471, 462, 462, 470, 470, 496, 496, 530, 530, 555, 555, 563, 563, 563, 563, 529, 529, 465, 465, 451, 485, 485, 485, 491, 491, 501, 501, 508, 508, 532, 532, 550, 550, 562, 532, 559, 559, 603, 603, 650, 650, 7

[113, 20, 20, 77, 77, 146, 146, 344, 344, 531, 531, 602, 602, 684, 684, 778, 778, 861, 861, 899, 899, 899, 899, 833, 833, 792, 792, 702, 702, 647, 647, 591, 591, 452, 452, 286, 286, 217, 217, 173, 173, 151, 151, 121, 121, 67, 67, 67, 94, 95, 95, 95, 95, 92, 92, 92, 92, 91, 91, 90, 90, 85, 85, 77, 77, 79, 79, 133, 133, 160, 160, 284, 284, 332, 332, 359, 359, 377, 377, 391, 391, 390, 390, 386, 386, 367, 367, 277, 277, 253, 253, 244, 244, 261, 261, 266]
[107, 170, 170, 207, 207, 268, 268, 372, 372, 461, 461, 563, 563, 630, 630, 704, 704, 771, 771, 873, 873, 967, 967, 1010, 1010, 1041, 1041, 1079, 1079, 1087, 1087, 1090, 1090, 1087, 1087, 1050, 1050, 970, 970, 900, 900, 836, 836, 687, 687, 389, 389, 300, 300, 159, 159, 24, 24, 16, 16, 18, 18, 33, 751, 768, 768, 815, 815, 849, 849, 857, 857, 834, 834, 791, 791, 780, 780, 778, 778, 787, 787, 800, 800, 820, 820, 867, 867, 849, 849, 827, 827, 850, 850, 883, 883, 898, 898, 899, 899, 890, 890, 890, 890, 882, 882, 864, 864, 851, 851, 840, 840, 83

[145, 152, 152, 262, 262, 361, 361, 474, 474, 541, 541, 592, 592, 806, 806, 809, 809, 808, 808, 801, 801, 747, 747, 709, 709, 667, 667, 619, 619, 547, 547, 488, 488, 415, 415, 300, 300, 236, 236, 212, 212, 198, 198, 188, 188, 166, 166, 162, 26, 29, 29, 58, 58, 89, 89, 142, 142, 173, 173, 243, 243, 285, 285, 314, 314, 340, 340, 392, 392, 429, 429, 456, 456, 495, 495, 514, 514, 553, 553, 650, 650, 691, 691, 729, 729, 832, 832, 876, 876, 940, 940, 962, 202, 216, 216, 254, 254, 307, 307, 338, 338, 372, 372, 556, 556, 597, 597, 631, 631, 664, 664, 691, 691, 713, 713, 820, 820, 857]
[581, 524, 524, 483, 483, 441, 441, 364, 364, 319, 319, 249, 249, 207, 207, 124, 124, 82, 82, 46, 46, 23, 23, 28, 28, 117, 117, 183, 183, 250, 250, 338, 338, 419, 419, 481, 481, 516, 516, 525, 514, 526, 526, 527, 527, 523, 523, 513, 513, 504, 521, 522, 522, 530, 551, 552, 552, 553, 553, 556, 556, 555, 555, 553, 553, 551, 551, 544, 544, 545, 545, 547, 547, 547, 547, 547, 547, 548, 548, 550, 550, 548, 548, 551, 551

[82, 58, 58, 63, 63, 88, 88, 102, 102, 126, 126, 165, 165, 279, 279, 352, 352, 446, 446, 477, 477, 523, 523, 581, 581, 584, 584, 577, 577, 559, 559, 499, 499, 416, 416, 323, 323, 234, 234, 136, 136, 95, 95, 77, 77, 50, 430, 420, 420, 407, 407, 382, 382, 371, 371, 379, 379, 435, 435, 464, 464, 572, 572, 593, 593, 601, 601, 593, 593, 585, 78, 76, 76, 79, 79, 84, 84, 95, 95, 115, 115, 136, 136, 143, 143, 143, 143, 138, 138, 134, 134, 117, 117, 109, 109, 84, 84, 77, 77, 77, 77, 106, 106, 117, 117, 122, 122, 107, 107, 87, 87, 79, 79, 90, 90, 108, 108, 114, 114, 104, 104, 86, 86, 72, 72, 48, 48, 48, 48, 63, 63, 79, 79, 90, 90, 94, 94, 87, 87, 69, 69, 56, 56, 47, 47, 54, 54, 66, 66, 82, 82, 88, 88, 90, 90, 81, 81, 65, 65, 53, 53, 59, 59, 78, 78, 83, 83, 72, 72, 54, 54, 38, 38, 25, 25, 33, 33, 50, 50, 94, 94, 112, 112, 119, 119, 111, 111, 92, 92, 58, 58, 46, 46, 42, 42, 55, 55, 72, 72, 82, 82, 82, 82, 65, 65, 50, 50, 35, 35, 22, 22, 32, 32, 55, 55, 64, 64, 72, 72, 79, 79, 65, 65, 47, 47, 32, 3

...

[199, 193, 193, 237, 237, 303, 303, 354, 354, 490, 490, 563, 563, 676, 676, 759, 759, 804, 804, 862, 862, 949, 949, 1002, 1002, 1026, 1026, 1039, 1039, 1047, 1047, 1043, 1043, 1031, 1031, 974, 974, 548, 548, 367, 367, 247, 247, 162, 162, 43, 43, 15, 15, 72, 72, 247, 247, 298, 298, 344, 344, 387, 458, 459, 459, 460, 460, 471, 471, 548, 548, 591, 591, 617, 617, 684, 684, 724, 724, 757, 757, 755, 755, 725]
[20, 86, 86, 207, 207, 378, 378, 500, 500, 554, 554, 629, 629, 685, 685, 676, 676, 652, 652, 625, 625, 513, 513, 437, 437, 249, 249, 180, 180, 123, 123, 86, 86, 62, 62, 51, 398, 402, 402, 413, 413, 427, 427, 462, 462, 492, 492, 510, 510, 528, 528, 560, 359, 364, 364, 372, 372, 384, 384, 400, 400, 418, 418, 436, 436, 441, 441, 437, 437, 426, 433, 431, 431, 435, 435, 435, 435, 449, 449, 456, 456, 457, 460, 464, 464, 472]
[20, 20, 20, 45, 45, 82, 82, 208, 208, 291, 291, 354, 354, 405, 405, 482, 482, 517, 517, 573, 573, 596, 596, 641, 641, 669, 669, 679, 679, 698, 698, 725, 725, 759, 759, 7

[484, 432, 432, 410, 410, 365, 365, 301, 301, 216, 216, 199, 199, 179, 179, 155, 155, 150, 150, 155, 155, 210, 210, 263, 263, 323, 323, 388, 388, 471, 471, 543, 543, 603, 603, 644, 644, 678, 678, 696, 696, 703, 703, 703, 703, 696, 696, 647, 647, 612, 612, 557, 557, 518, 518, 481, 342, 334, 334, 322, 322, 300, 300, 286, 286, 272, 272, 265, 265, 266, 266, 273, 273, 302, 302, 325, 325, 327, 143, 137, 137, 136, 136, 145, 145, 148, 148, 160, 160, 185, 185, 214, 214, 251, 251, 309, 309, 343, 343, 374, 374, 395, 395, 414, 414, 419, 419, 419, 419, 413, 413, 387, 387, 363, 363, 338, 158, 124, 124, 103, 103, 54, 54, 42, 42, 30, 30, 28, 28, 58, 58, 84, 84, 109, 109, 133, 133, 156, 156, 181, 181, 188, 188, 188, 188, 180, 180, 154, 154, 137, 137, 134, 134, 142, 142, 156, 156, 193, 193, 222, 222, 262, 262, 281, 281, 285, 285, 280, 280, 245, 245, 210, 210, 198, 198, 192, 192, 190, 190, 195, 195, 215, 215, 226, 226, 235, 235, 237, 237, 193, 193, 160, 160, 142, 142, 142, 142, 152, 152, 162, 162, 177]
[

[92, 190, 71, 159, 159, 342, 342, 417, 57, 96, 96, 126, 126, 168, 168, 308, 308, 356, 356, 401, 401, 433, 393, 443, 443, 457, 438, 452, 452, 487, 487, 503, 503, 503, 503, 489, 489, 469, 469, 455, 474, 490, 490, 501, 501, 515, 515, 528, 528, 530, 530, 523, 523, 515, 515, 514, 15, 20, 20, 37, 37, 56, 56, 69, 69, 85, 85, 105, 105, 129, 129, 147, 147, 186, 186, 203, 203, 219, 219, 240, 240, 271, 271, 331, 331, 363, 363, 397, 397, 414, 414, 447, 447, 493, 493, 519, 519, 546, 546, 577]
[107, 146, 146, 174, 174, 241, 241, 298, 298, 421, 421, 512, 512, 558, 558, 568, 568, 565, 565, 558, 558, 517, 517, 481, 481, 446, 446, 371, 371, 331, 331, 289, 289, 242, 242, 202, 202, 178, 178, 147, 147, 131, 131, 117, 117, 106, 357, 355, 355, 376, 376, 365, 365, 322, 322, 317, 317, 325, 325, 382, 382, 412, 412, 421, 421, 418, 313, 300, 300, 259, 259, 294, 294, 305, 305, 330, 330, 337, 337, 334, 328, 293, 293, 279, 279, 264, 264, 232, 232, 231, 231, 237, 237, 265, 265, 293, 293, 348, 348, 358, 358, 345, 345,

[20, 44, 44, 61, 61, 107, 107, 274, 274, 413, 413, 487, 487, 569, 569, 644, 644, 712, 712, 734, 734, 750, 750, 771, 771, 770, 770, 768, 768, 763, 763, 730, 730, 452, 452, 395, 395, 344, 344, 92, 92, 41, 506, 509, 509, 533, 533, 557, 557, 585, 585, 607, 607, 642, 642, 640, 640, 626, 626, 606, 606, 588, 588, 552, 552, 543, 543, 545, 545, 562, 562, 590, 590, 603, 603, 611, 611, 615, 615, 613, 613, 605, 605, 590, 590, 582, 582, 577, 577, 584, 584, 595, 595, 606, 606, 604, 604, 586, 586, 577, 577, 581, 581, 599, 599, 621, 621, 632, 632, 637, 637, 628, 628, 610, 610, 595, 595, 599, 599, 618, 618, 634, 634, 643, 643, 635, 635, 608, 608, 591, 591, 583, 583, 594, 594, 609, 609, 622, 622, 623, 623, 615, 615, 599, 599, 586, 586, 604, 604, 629, 308, 331, 331, 412, 412, 469, 469, 517, 517, 553, 553, 555, 555, 558, 558, 552, 214, 214, 214, 241, 241, 252, 252, 276, 276, 303, 303, 349, 349, 416, 416, 575, 575, 629, 629, 653, 653, 688, 688, 713, 713, 725, 725, 733, 733, 737, 737, 731, 731, 720, 720, 70

ConnectionError: HTTPConnectionPool(host='54.82.94.146', port=80): Max retries exceeded with url: /Hdata (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1a3590ebd0>: Failed to establish a new connection: [Errno 50] Network is down',))

In [3]:
num_of_boats = 1
count = 0
for i in range(num_of_boats):
    output = []
    generate_result = generate()
    print(generate_result)
    result = get_sketch(generate_result)
    print(result)
    total = len(result)
    for i in result:
        entry = []
        if i[2] != 0:
            pen = 0
        else:
            if i[3] == 1:
                pen = i[3]
            else:
                pen = i[4]
        if count == total -2:
            pen = 0
        entry.extend(i[:2])
        entry.append(pen)
        output.append(entry)
        count += 1
    #print(output)

[[4.9591896359217724, 20.325722968801127, 1, 0, 0], [16.545484171148235, 6.514362763131369, 1, 0, 0], [29.547741578560487, 3.3517989192580675, 1, 0, 0], [67.632119743842523, -4.8609568718228866, 1, 0, 0], [99.817363592974402, -26.496335899956037, 1, 0, 0], [83.183665142034684, -32.836387364689543, 1, 0, 0], [22.470108262402277, -9.5282629910388312, 1, 0, 0], [14.302584586234294, -10.314495385543617, 1, 0, 0], [23.628350253901189, -13.323085451835533, 1, 0, 0], [63.97669774616044, -36.79800537550809, 1, 0, 0], [51.109516092544958, -11.655412365530591, 1, 0, 0], [37.551795003837825, -8.7079340857432683, 1, 0, 0], [22.851514814320314, -3.7436380633604505, 1, 0, 0], [66.130937921323948, -3.4787812543702397, 1, 0, 0], [31.770123430506366, -0.20593941344439479, 1, 0, 0], [68.842820227363319, 13.706555187571713, 1, 0, 0], [77.60498602150011, 25.459598916806957, 1, 0, 0], [63.609942413118056, 13.935482103264986, 1, 0, 0], [178.69554147086131, 7.5821855140696615, 1, 0, 0], [51.364438930662097, 